считываем данные из student-mat.csv

In [58]:
import pandas as pd
import numpy as np
import math

csv_file_path = "student-mat.csv"

df = pd.read_csv(csv_file_path, sep=";")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      395 non-null    object
 1   sex         395 non-null    object
 2   age         395 non-null    int64 
 3   address     395 non-null    object
 4   famsize     395 non-null    object
 5   Pstatus     395 non-null    object
 6   Medu        395 non-null    int64 
 7   Fedu        395 non-null    int64 
 8   Mjob        395 non-null    object
 9   Fjob        395 non-null    object
 10  reason      395 non-null    object
 11  guardian    395 non-null    object
 12  traveltime  395 non-null    int64 
 13  studytime   395 non-null    int64 
 14  failures    395 non-null    int64 
 15  schoolsup   395 non-null    object
 16  famsup      395 non-null    object
 17  paid        395 non-null    object
 18  activities  395 non-null    object
 19  nursery     395 non-null    object
 20  higher    

для облегчения подсчетов переводим в матричный вил и разделяем на шкалы порядка, наименований и абсолютную шкалу

In [59]:
matrix_df = df.values.T

numeric_df = []
categorical_df = []
absolute_df = ['age', 'absences']

for column in df.columns:
    if df[column].dtype in ['int64', 'float64'] and column not in absolute_df:
        numeric_df.append(column)
    elif column not in absolute_df:
        categorical_df.append(column)

num_rows_df = df.shape[0]
print(f"Количество строк датасета: {num_rows_df}")

num_columns_df = df.shape[1]
print(f"Количество столбцов датасета: {num_columns_df}")

num_columns_numeric_df = len(numeric_df)
print(f"Данные с шкалой порядка (количество столбцов: {num_columns_numeric_df})")
print(numeric_df)

num_columns_categorical_df = len(categorical_df)
print(f"\nДанные с шкалой наименований (количество столбцов: {num_columns_categorical_df})")
print(categorical_df)

num_columns_absolute_df = len(absolute_df)
print(f"\nДанные с абсолютной шкалой (количество столбцов: {num_columns_absolute_df})") 
print(absolute_df)


Количество строк датасета: 395
Количество столбцов датасета: 33
Данные с шкалой порядка (количество столбцов: 14)
['Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'G1', 'G2', 'G3']

Данные с шкалой наименований (количество столбцов: 17)
['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']

Данные с абсолютной шкалой (количество столбцов: 2)
['age', 'absences']


считываем сколько у нас строк в датафрейме(количество объектов), формируем матрицу попарных расстояний с помощью метрик
для наменовний:
d(i,l) = 0, если (Xi = X и Xl = X) или (Xi = X И Xl = X)
d(i,l) = 1, если (Xi = X и Xl ≠ X) или (Xi ≠ X и Xl = X)

для порядка:
d(i,l) = 0, если (Xi > X и Xl > X) или (Xi < X и Xl < X) или (Xi = X и Xl = X)
d(i,l) = 1, если (Xi > X и Xl < X) или (Xi < X И Xl > X)
d(i,l) = 0.5, если (Xi >< X и Xl = X) или (Xi = X И Xl >< X)

для абсолютной:
d(i,l) = 0, если |(Xi - Xl)| / (X.max - X.min)

Пояснение, X пробегает все значения в столбце, а Xi это значение 1 объекта, Xl значени 2 объекта

Так как расстояния должны быть [0; 1], то нормируем их деля на количество всех объектов

В конце по формуле из группы считаем расстояние между объектами

In [81]:
matrix_pairwise_distances_df = np.zeros((num_rows_df, num_rows_df), dtype=np.float64)

def calc_categorical(i, l , column):
    d = np.float64(0)
    xi = column[i]
    xl = column[l]

    for x in column:
        if(((xi == x) and (xl == x)) or ((xi != x) and (xl != x))):
            d += 0
        else:
            d += 1

    d = d / np.float64(num_rows_df)  
    return d

def calc_numeric(i, l , column):
    d = np.float64(0)
    xi = column[i]
    xl = column[l]

    for x in column:
        if(((xi > x) and (xl > x)) or ((xi < x) and (xl < x)) or ((xi == x) and (xl == x))):
            d += 0
        elif(((xi > x) and (xl < x)) or ((xi < x) and (xl > x))):
            d += 1
        else:
            d += 0.5

    d = d / np.float64(num_rows_df)  
    return d

def calc_absolute(i, l , column):
    d = np.float64(0)
    xi = column[i]
    xl = column[l]

    d = abs(xi - xl) / (np.float64(max(column)) - np.float64(min(column)))  
    return d

for i in range(0, num_rows_df - 1):
    for l in range(i + 1, num_rows_df):
        t = 0
        d = np.float64(0)

        for column in df.columns:
            if(column in categorical_df):
                d += math.pow(calc_categorical(i, l, matrix_df[t]), 2)
            elif(column in absolute_df):
                d += math.pow(calc_absolute(i, l, matrix_df[t]), 2)
            else:
                d += math.pow(calc_numeric(i, l, matrix_df[t]), 2)
            t += 1
        
        matrix_pairwise_distances_df[i][l] = math.sqrt(d) / math.sqrt(num_columns_df)
        matrix_pairwise_distances_df[l][i] = matrix_pairwise_distances_df[i][l]

pairwise_distances_df = pd.DataFrame(matrix_pairwise_distances_df)
csv_file_path = "matrix_pairwise_distances_df.csv"
pairwise_distances_df.to_csv(csv_file_path, sep=';', index=False)



тут будет проверка на компактность